In [38]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar_template as lc
from tensorflow.contrib.layers import flatten
import matplotlib.pyplot as plt
import time

from tqdm import tqdm
import scipy.io as spio
import pickle
import os
import cv2
import timeit
from skimage.transform import rescale, resize, downscale_local_mean

In [39]:
#fetch the data

def fetch_data(directory, label): 
    item_list = []
    step = 0
    for file in tqdm(os.listdir(directory)):
        full_img_str = directory + "/" + file
        #print(full_img_str)

        mat = spio.loadmat(full_img_str, squeeze_me=True)
        data = np.abs(mat["data_store"])
        smaller_data = data[200:300]
        
        image_resized = resize(smaller_data, (100, 320, 4),
                       anti_aliasing=True)
        
        sub_list = []
        for i in range(8):
            chunk = image_resized[:,40*i:40*i+40]
            sub_list.append(chunk)
        
        #
        #print(data.shape)
        #print(data)
        ###append the img and label to the list###
        #sub_list = [image_resized, label]
        #print(sub_list)
        item_list.append([sub_list, label])
        
    return item_list
    
bike1 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1000/data", [1, 0, 0])
bike2 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1001/data", [1, 0, 0])
bike3 = fetch_data("Desktop/ee596prepro/2019_04_09_bms1002/data", [1, 0, 0])

car1 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1000/data", [0, 1, 0])
car2 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1001/data", [0, 1, 0])
car3 = fetch_data("Desktop/ee596prepro/2019_04_09_cms1002/data", [0, 1, 0])

ped1 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1000/data", [0, 0, 1])
ped2 = fetch_data("Desktop/ee596prepro/2019_04_09_pms1001/data", [0, 0, 1])
ped3 = fetch_data("Desktop/ee596prepro/2019_04_09_pms2000/data", [0, 0, 1])



#directory = "Desktop/ee596prepro/2019_04_09_bms1000/data"

100%|████████████████████████████████████████████████████████████████████████████████| 333/333 [00:17<00:00, 19.49it/s]


In [44]:
def mini_batch(features,labels,mini_batch_size):
    """
    Args:
        features: features for one batch
        labels: labels for one batch
        mini_batch_size: the mini-batch size you want to use.
    Hint: Use "yield" to generate mini-batch features and labels
    """
    #split the data into batches
    amount_of_data = len(features)
    number_of_bunches = amount_of_data/mini_batch_size
    
    bunches_features = []
    bunches_labels = []
    
    #loop over breaking the data into batches
    for i in range(int(number_of_bunches)):
        current_range = i * mini_batch_size
        f_b = features[current_range:current_range+mini_batch_size]
        l_b = labels[current_range:current_range+mini_batch_size]
        
        bunches_features.append(f_b)
        bunches_labels.append(l_b)
    
    #return the mini-batched data
    return bunches_features, bunches_labels

#full_list = bike1 + bike2 + bike3 + car1 + car2 + car3 + ped1 + ped2 + ped3
full_list = bike1 + car1 + ped1 + bike2 + car2 + ped2 + bike3 + car3 + ped3
#test_list = bike3 + car3 + ped3


#test_list = bike3 + car3 + ped3

np.random.shuffle(full_list)
#np.random.shuffle(test_list)
print(np.asarray(full_list).shape)
#print(np.asarray(full_list)[0][0].shape)
#np.random.shuffle(test_list)

training_set = full_list[0:2100]
valid_set = full_list[2100:2550]
test_set = full_list[2550:2997]
#full_list = None
#bike1 = None
#car1 = None
#ped1 = None

train_set_data = np.zeros((len(training_set), 8, 100, 40, 4))
train_set_labels = np.zeros((len(training_set), 3))
valid_set_data = np.zeros((len(valid_set), 8, 100, 40, 4))
valid_set_labels = np.zeros((len(valid_set), 3))
test_set_data = np.zeros((len(test_set), 8, 100, 40, 4))
test_set_labels = np.zeros((len(test_set), 3))
#print(np.asarray(training_set).shape)
#print(training_set[0][0][0])
#print(training_set[0][:][1])
#print(training_set[0][:][0].shape)
#print(training_set[0][:][0])
#split into training, valid, and testing
for i in range(len(training_set)):
    train_set_data[i] = training_set[i][0]#np.reshape(training_set[i][0], (100, 960, 4))
    train_set_labels[i] = training_set[i][1]
    
for i in range(len(valid_set)):
    valid_set_data[i] = valid_set[i][0]#np.reshape(valid_set[i][0], (100, 960, 4))
    valid_set_labels[i] = valid_set[i][1]

for i in range(len(test_set)):
    test_set_data[i] = test_set[i][0]#np.reshape(test_set[i][0], (100, 960, 4))
    test_set_labels[i] = test_set[i][1]

#print(np.asarray(train_set_data).shape)
#print(train_set_data.shape)


#train_set_data, train_set_labels = mini_batch(train_set_data,train_set_labels,5)
#valid_set_data, valid_set_labels = mini_batch(valid_set_data,valid_set_labels,5)
#test_set_data, test_set_labels = mini_batch(test_set_data,test_set_labels,5)

#training_set = None
#valid_set = None
#test_set = None


#test_list = None

(2997, 2)


In [45]:
print(test_set_data[0].shape)
print(train_set_labels)

(8, 100, 40, 4)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [46]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#import keras.models
#import keras.layers
from tensorflow.keras import metrics
#X = seq.reshape(len(seq), 1, 1)
#y = seq.reshape(len(seq), 1)
seq_imgs = 25
print(np.asarray(train_set_data).shape)
#train_set_data = np.reshape(train_set_data, (980*25, 227, 227, 3))
#print(train_set_labels.shape)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(30, (5, 5), activation="relu"),input_shape=(8, 100, 40, 4)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D(2, (2, 2))))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
model.add(tf.keras.layers.LSTM(50))
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))

adam_opt = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#7
#sgd_opt = tf.keras.optimizers.SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=adam_opt,
              metrics=[metrics.mae, metrics.categorical_accuracy])

model.fit(train_set_data, train_set_labels, epochs=10, batch_size=5, verbose = 2, validation_data = (valid_set_data, valid_set_labels))



(2100, 8, 100, 40, 4)
Train on 2100 samples, validate on 450 samples
Epoch 1/10
 - 24s - loss: 0.8492 - mean_absolute_error: 0.3576 - categorical_accuracy: 0.6071 - val_loss: 0.7144 - val_mean_absolute_error: 0.3030 - val_categorical_accuracy: 0.7156
Epoch 2/10
 - 22s - loss: 0.5751 - mean_absolute_error: 0.2501 - categorical_accuracy: 0.7933 - val_loss: 0.5167 - val_mean_absolute_error: 0.2228 - val_categorical_accuracy: 0.7800
Epoch 3/10
 - 22s - loss: 0.3998 - mean_absolute_error: 0.1821 - categorical_accuracy: 0.8633 - val_loss: 0.3728 - val_mean_absolute_error: 0.1640 - val_categorical_accuracy: 0.8689
Epoch 4/10
 - 22s - loss: 0.2870 - mean_absolute_error: 0.1343 - categorical_accuracy: 0.9086 - val_loss: 0.2774 - val_mean_absolute_error: 0.1274 - val_categorical_accuracy: 0.9156
Epoch 5/10
 - 22s - loss: 0.2198 - mean_absolute_error: 0.1055 - categorical_accuracy: 0.9371 - val_loss: 0.2205 - val_mean_absolute_error: 0.1054 - val_categorical_accuracy: 0.9333
Epoch 6/10
 - 22s - l

In [47]:
acc = 0

test_set_data = np.asarray(test_set_data)
test_set_labels = np.asarray(test_set_labels)

#fetch batch
#batch_x = test_set_data[k]
#batch_y = test_set_labels[k]
#run optimization
prediction = model.predict(test_set_data, verbose=0)
#print(prediction)
for k in range(len(test_set_data)):
    if (np.argmax(prediction[k]) == np.argmax(test_set_labels[k])):
        acc += 1
    else:
        acc += 0
    



acc = acc/len(test_set_data)
print("Test Accuracy= {:.3f}".format(acc))

Test Accuracy= 0.966
